# Step 3: Model Operationalization & Deployment

In the previous script, you learned how to save lstm trained models to files. You also learned that model weights are easily stored using  HDF5 format and that the network structure can be saved in JSON. In this script, you will learn how to load your models up, operationalize and use them to make future predictions.

In [82]:
import keras
# import the libraries
import os
import pandas as pd
import numpy as np
import json
import shutil
from keras.models import model_from_json

import h5py

# For creating the deployment schema file
from azureml.api.schema.dataTypes import DataTypes
from azureml.api.schema.sampleDefinition import SampleDefinition
from azureml.api.realtime.services import generate_schema

# Use the Azure Machine Learning data collector to log various metrics
from azureml.logging import get_azureml_logger
run_logger = get_azureml_logger()
run_logger.log('amlrealworld.predictivemaintenanceforpm.operationalization','true')

# For Azure blob storage access
from azure.storage.blob import BlockBlobService
from azure.storage.blob import PublicAccess

## Model storage 

We will stor the model in an Azure Blob Storage Container for easy retreival to your deployment platform. 
Instructions for setting up your Azure Storage account are available within this link (https://docs.microsoft.com/en-us/azure/storage/blobs/storage-python-how-to-use-blob-storage). You will need to copy your account name and account key from the _Access Keys_ area in the portal into the following code block. These credentials will be reused in all four Jupyter notebooks.

We will handle creating the containers and writing the data to these containers for each notebook. Further instructions for using Azure Blob storage with AML Workbench are available (https://github.com/Azure/ViennaDocs/blob/master/Documentation/UsingBlobForStorage.md).

You will need to enter the **ACCOUNT_NAME** as well as the **ACCOUNT_KEY** in order to access Azure Blob storage account you have created. This notebook will create and store all the resulting data files in a blob container under this account.


In [83]:
# Enter your Azure blob storage details here 
ACCOUNT_NAME = "<your blob storage account name>"

# You can find the account key under the _Access Keys_ link in the 
# [Azure Portal](portal.azure.com) page for your Azure storage container.
ACCOUNT_KEY = "<your blob storage account key>"

#-------------------------------------------------------------------------------------------
# The data from the Data Ingestion and Preparation notebook is stored in the sensordata ingestion container.
MODEL_CONTAINER = "pmlstmmodel" 
# Connect to your blob service     
az_blob_service = BlockBlobService(account_name=ACCOUNT_NAME, account_key=ACCOUNT_KEY)

In [84]:
# We will store each of these data sets in a local persistance folder
SHARE_ROOT = os.environ['AZUREML_NATIVE_SHARE_DIRECTORY']

# These file names detail the data files. 
TEST_DATA = 'PM_test_files.pkl'

# We'll serialize the model in json format
LSTM_MODEL = 'modellstm.json'

# and store the weights in h5
MODEL_WEIGHTS = 'modellstm.h5'

# and the schema file
SCHEMA_FILE = 'service_schema.json'

## Load the test data frame

We have previously stored the test data frame in the local persistance directory indicated by the SHARE_ROOT variable. We'll use this data frame to test the model deployment and build the model schema to describe the deployment function calls.

In [85]:
test_df = pd.read_pickle(SHARE_ROOT + TEST_DATA)
test_df.head(10)

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,cycle_norm,RUL,label1,label2
0,1,1,0.632184,0.750000,0.0,0.0,0.545181,0.310661,0.269413,0.0,...,0.0,0.333333,0.0,0.0,0.558140,0.661834,0.000000,142,0,0
1,1,2,0.344828,0.250000,0.0,0.0,0.150602,0.379551,0.222316,0.0,...,0.0,0.416667,0.0,0.0,0.682171,0.686827,0.002770,141,0,0
2,1,3,0.517241,0.583333,0.0,0.0,0.376506,0.346632,0.322248,0.0,...,0.0,0.416667,0.0,0.0,0.728682,0.721348,0.005540,140,0,0
3,1,4,0.741379,0.500000,0.0,0.0,0.370482,0.285154,0.408001,0.0,...,0.0,0.250000,0.0,0.0,0.666667,0.662110,0.008310,139,0,0
4,1,5,0.580460,0.500000,0.0,0.0,0.391566,0.352082,0.332039,0.0,...,0.0,0.166667,0.0,0.0,0.658915,0.716377,0.011080,138,0,0
5,1,6,0.568966,0.750000,0.0,0.0,0.271084,0.176150,0.217421,0.0,...,0.0,0.333333,0.0,0.0,0.596899,0.624827,0.013850,137,0,0
6,1,7,0.500000,0.666667,0.0,0.0,0.271084,0.268149,0.381330,0.0,...,0.0,0.250000,0.0,0.0,0.550388,0.691798,0.016620,136,0,0
7,1,8,0.534483,0.500000,0.0,0.0,0.400602,0.214737,0.314652,0.0,...,0.0,0.416667,0.0,0.0,0.705426,0.591273,0.019391,135,0,0
8,1,9,0.293103,0.500000,0.0,0.0,0.201807,0.485066,0.506921,0.0,...,0.0,0.250000,0.0,0.0,0.744186,0.770367,0.022161,134,0,0
9,1,10,0.356322,0.416667,0.0,0.0,0.259036,0.309789,0.276671,0.0,...,0.0,0.250000,0.0,0.0,0.565891,0.673571,0.024931,133,0,0


In [86]:
# pick the feature columns 
# Sequence help order the observations in "time"
sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']

# key columns group the machines
key_cols = ['id', 'cycle']

# Labels are what we're predicting.
label_cols = ['label1', 'label2', 'RUL']

# The scoreing data should not have labels... if we knew the label, 
# we wouldn'y need to predict.
score_df = test_df.drop(label_cols, axis = 1)


### Test init() and run() functions to read from the working directory

In [87]:
def init():
    # read in the model file
    from keras.models import model_from_json
    global loaded_model
    
    # load json and create model
    with open(SHARE_ROOT + LSTM_MODEL, 'r') as json_file:
        loaded_model_json = json_file.read()
        json_file.close()
        loaded_model = model_from_json(loaded_model_json)
    
    # load weights into new model
    loaded_model.load_weights(os.path.join(SHARE_ROOT, MODEL_WEIGHTS))


In [88]:
def run(score_input): 
    # Create the sequences
    sequence_length = 50
    sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']
    key_cols = ['id', 'cycle']

    input_features = score_input.columns.values.tolist()
    sensor_cols = [x for x in input_features if x not in set(key_cols)]
    sensor_cols = [x for x in sensor_cols if x not in set(sequence_cols)]

    # The time is sequenced along
    # This may be a silly way to get these column names, but it's relatively clear
    sequence_cols.extend(sensor_cols)
    
    seq_array = [score_input[score_input['id']==id][sequence_cols].values[-sequence_length:]
                 for id in score_input['id'].unique() if len(score_input[score_input['id']==id]) >= sequence_length]

    seq_array = np.asarray(seq_array).astype(np.float32)
    try:
        prediction = loaded_model.predict_proba(seq_array)
        #print(prediction)
        
        return (prediction)
    except Exception as e:
        return(str(e))

In [89]:
init()

In [90]:
prb=run(score_df)
print(prb.shape)
print(prb)

(93, 1)
[[2.8877859e-03]
 [1.3711210e-03]
 [2.1243060e-03]
 [1.5255641e-03]
 [1.0095595e-03]
 [1.5838079e-03]
 [1.4917494e-03]
 [1.4009204e-03]
 [1.5001412e-03]
 [1.0851335e-03]
 [1.3480394e-03]
 [9.0019830e-04]
 [1.0240693e-03]
 [5.0882497e-03]
 [1.3305876e-01]
 [1.2116296e-03]
 [8.8789827e-01]
 [1.2445084e-03]
 [7.5811712e-04]
 [8.7095404e-01]
 [8.6192781e-04]
 [2.0066202e-03]
 [1.2240123e-03]
 [1.5637396e-03]
 [1.5616176e-03]
 [9.8309302e-01]
 [4.4359355e-03]
 [1.2643852e-03]
 [9.7795552e-01]
 [9.8700225e-01]
 [6.1374915e-01]
 [1.6415939e-01]
 [3.1632520e-03]
 [2.5539070e-01]
 [4.8845494e-01]
 [9.5267975e-01]
 [2.1306502e-03]
 [1.5662438e-03]
 [2.2981316e-03]
 [1.3850127e-02]
 [1.5890056e-03]
 [1.4705922e-03]
 [5.5022943e-01]
 [1.1768042e-03]
 [1.8632254e-03]
 [8.5451730e-02]
 [4.9792055e-02]
 [7.3846581e-04]
 [1.0360088e-03]
 [9.4570303e-01]
 [1.6720097e-03]
 [7.4768290e-03]
 [7.3503749e-04]
 [1.7222582e-03]
 [1.8618371e-01]
 [6.8073156e-03]
 [3.9924150e-03]
 [7.7436082e-02]
 [6.15

In [91]:
print(score_df.id.unique())

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100]


In [92]:
tst_df=score_df.loc[score_df['id'] == 1]

print(tst_df.shape)

# Because 
run(tst_df)

(31, 27)


'Error when checking : expected lstm_1_input to have 3 dimensions, but got array with shape (0, 1)'

In [93]:
tst_df=score_df.loc[score_df['id'] == 3]

print(tst_df.shape)

# Because 
run(tst_df)

(126, 27)


array([[0.00288779]], dtype=float32)

## Persist model assets

Next we persist the assets we have created to disk for use in operationalization.

In [94]:
# define the input data frame
inputs = {"score_input": SampleDefinition(DataTypes.PANDAS, score_df)}

json_schema = generate_schema(run_func=run, inputs=inputs, filepath=SCHEMA_FILE)

# save the schema file for deployment
out = json.dumps(json_schema)
with open(SHARE_ROOT + SCHEMA_FILE, 'w') as f:
    f.write(out)

In [95]:
%%writefile {SHARE_ROOT}/webservices_conda.yaml

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for managed runs. These include runs against
# the localdocker, remotedocker, and cluster compute targets.

# Note that this file is NOT used to automatically manage dependencies for the
# local compute target. To provision these dependencies locally, run:
# conda env update --file conda_dependencies.yml

# Details about the Conda environment file format:
# https://conda.io/docs/using/envs.html#create-environment-file-by-hand

# For managing Spark packages and configuration, see spark_dependencies.yml.

name: project_environment
channels:
- conda-forge
- defaults
dependencies:
  - python=3.5.2
  - pip:
    - azure-common==1.1.8
    - azure-storage==0.36.0
    - numpy==1.14.0 
    - sklearn
    - keras
    - tensorflow
    - h5py

Overwriting /azureml-share//webservices_conda.yaml


In [96]:
%%writefile {SHARE_ROOT}/lstmscore.py

# import the libraries
import keras
import json
import shutil
import numpy as np


def init():
    # read in the model file
    from keras.models import model_from_json
    global loaded_model
    
    # load json and create model
    with open('modellstm.json', 'r') as json_file:
        loaded_model_json = json_file.read()
        json_file.close()
        loaded_model = model_from_json(loaded_model_json)
    
    # load weights into new model
    loaded_model.load_weights("modellstm.h5")

def run(score_input):
    # Create the sequences
    sequence_length = 50
    sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']
    key_cols = ['id', 'cycle']

    input_features = score_input.columns.values.tolist()
    sensor_cols = [x for x in input_features if x not in set(key_cols)]
    sensor_cols = [x for x in sensor_cols if x not in set(sequence_cols)]

    # The time is sequenced along
    # This may be a silly way to get these column names, but it's relatively clear
    sequence_cols.extend(sensor_cols)
    
    seq_array = [score_input[score_input['id']==id][sequence_cols].values[-sequence_length:] 
                 for id in score_input['id'].unique() if len(score_input[score_input['id']==id]) >= sequence_length]

    seq_array = np.asarray(seq_array).astype(np.float32)
    try:
        prediction = loaded_model.predict_proba(seq_array)
        #print(prediction)
        
        return (prediction)
    except Exception as e:
        return(str(e))
    
if __name__ == "__main__":
    init()
    run("{\"score_df\": [{\"s20\": 0.5581395348837184, \"s10\": 0.0, \"s2\": 0.5451807228915584, \"s21\": 0.6618337475835432, \"s9\": 0.12761374854168395, \"s19\": 0.0, \"s3\": 0.31066056245912677, \"cycle\": 1, \"s15\": 0.3089649865332831, \"s4\": 0.2694125590817009, \"s1\": 0.0, \"s11\": 0.2083333333333357, \"cycle_norm\": 0.0, \"s13\": 0.2205882352941444, \"s5\": 0.0, \"s18\": 0.0, \"s8\": 0.2121212121210192, \"s14\": 0.1321601816492901, \"s6\": 1.0, \"setting2\": 0.75, \"setting1\": 0.632183908045977, \"s12\": 0.6460554371002161, \"s17\": 0.3333333333333357, \"s16\": 0.0, \"id\": 1, \"setting3\": 0.0, \"s7\": 0.6521739130434696}, {\"s20\": 0.6821705426356601, \"s10\": 0.0, \"s2\": 0.15060240963856586, \"s21\": 0.6868268434134208, \"s9\": 0.14668401687158195, \"s19\": 0.0, \"s3\": 0.37955090473076325, \"cycle\": 2, \"s15\": 0.21315890727203168, \"s4\": 0.2223160027008788, \"s1\": 0.0, \"s11\": 0.38690476190476275, \"cycle_norm\": 0.002770083102493075, \"s13\": 0.26470588235270043, \"s5\": 0.0, \"s18\": 0.0, \"s8\": 0.16666666666696983, \"s14\": 0.20476829394158358, \"s6\": 1.0, \"setting2\": 0.25, \"setting1\": 0.3448275862068965, \"s12\": 0.7398720682302695, \"s17\": 0.4166666666666714, \"s16\": 0.0, \"id\": 1, \"setting3\": 0.0, \"s7\": 0.8051529790660226}, {\"s20\": 0.7286821705426334, \"s10\": 0.0, \"s2\": 0.3765060240963862, \"s21\": 0.7213476940071786, \"s9\": 0.15808130664991182, \"s19\": 0.0, \"s3\": 0.34663178548071016, \"cycle\": 3, \"s15\": 0.4586379376683354, \"s4\": 0.3222484807562438, \"s1\": 0.0, \"s11\": 0.38690476190476275, \"cycle_norm\": 0.0055401662049861505, \"s13\": 0.2205882352941444, \"s5\": 0.0, \"s18\": 0.0, \"s8\": 0.22727272727297532, \"s14\": 0.15564041696769948, \"s6\": 1.0, \"setting2\": 0.5833333333333334, \"setting1\": 0.5172413793103449, \"s12\": 0.6993603411513902, \"s17\": 0.4166666666666714, \"s16\": 0.0, \"id\": 1, \"setting3\": 0.0, \"s7\": 0.6859903381642596}]}")

Overwriting /azureml-share//lstmscore.py


and generate the schema file

In [97]:
%%writefile {SHARE_ROOT}/test_service.py

import urllib
import json 
import requests
import pandas as pd

# The URL will need to be editted after service create.
url = 'http://127.0.0.1:32773/score'

## Sequence length will need to match the training sequence length from
## 2_model_building_and_evaluation.ipynb
sequence_length = 50

# We'll read in this data to test the service
test_df = pd.read_pickle('PM_test_files.pkl')

# Labels are what we're predicting.
label_cols = ['label1', 'label2', 'RUL']

# The scoreing data should not have labels... if we knew the label, 
# we wouldn't need to predict.
score_df = test_df.drop(label_cols, axis = 1)
headers = {'Content-Type':'application/json'}

# Now get the machine numbers, for each machine get the 
# prediction for the label timepoint
machineID = score_df['id'].unique()

for ind in machineID:
    
    try:
        body = score_df[score_df.id==ind]
        print('ID {}: {} \t {}'.format(ind, body.shape, body.tail(sequence_length+ 10).shape))
        body = "{\"score_input\": " + \
                    body.tail(sequence_length+10).to_json(orient="records") +\
                    "}"
        
        req = urllib.request.Request(url, str.encode(body), headers) 
        with urllib.request.urlopen(req) as response:
            the_page = response.read()
            print('ID {}: {}'.format(ind,the_page))
        
    except urllib.error.HTTPError as error:
        print("The request failed with status code {}: \n{}".format(error, error.read))

        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(error.reason)      

Overwriting /azureml-share//test_service.py


## Packaging

To move the model artifacts around, we'll zip them into one file. We can then retreive this file from the persistance shared folder on your DSVM.

https://docs.microsoft.com/en-us/azure/machine-learning/preview/how-to-read-write-files



In [98]:
# Compress the operationalization assets for easy blob storage transfer
# We can remove the persisted data files.
# !rm {SHARE_ROOT}/PM*.pkl
!rm {SHARE_ROOT}/PM_train_files.pkl
!ls {SHARE_ROOT}
MODEL_O16N = shutil.make_archive('LSTM_o16n', 'zip', SHARE_ROOT)

# Create a new container if necessary, otherwise you can use an existing container.
# This command creates the container if it does not already exist. Else it does nothing.
az_blob_service.create_container(MODEL_CONTAINER,
                                 fail_on_exist=False, 
                                 public_access=PublicAccess.Container)

# Transfer the compressed operationalization assets into the blob container.
az_blob_service.create_blob_from_path(MODEL_CONTAINER, "LSTM_o16n.zip", str(MODEL_O16N)) 

rm: cannot remove '/azureml-share//PM_train_files.pkl': No such file or directory
PM_test_files.pkl  modellstm.h5    service_schema.json	webservices_conda.yaml
lstmscore.py	   modellstm.json  test_service.py


# Deployment

Once the assets are stored, we can download them into a deployment compute context for operationalization on an Azure web service. For this scenario, we will deploy this on our local docker container context.

We demonstrate how to setup this web service this through a CLI window opened in the AML Workbench application. 

## Download the model

To download the model we've saved, follow these instructions on a local computer.

 - Open the Azure Portal
 - In the left hand pane, click on All resources
 - Search for the storage account using the name you provided earlier in this notebook.
 - Choose the storage account from search result list, this will open the storage account panel.
 - On the storage account panel, choose Blobs
 - On the Blobs panel choose the container `pmlstmmodel`
 - Select the file `LSTM_o16n.zip` and on the properties pane for that blob, choose download.

Once downloaded, unzip the file into the directory of your choosing. The zip file contains three deployment assets:

- the `lstmscore.py` file which contains functionst to do the model scoring
- the `modellstm.json` model definition file
- the `modellstm.h5` model weights file
- the `service_schema.json` which defines the input data schema

Additionally, because we are using both `keras` and `tensorflow` in this deployment, we will need to copy the `conda_dependencies.yaml` file from the `<project>\aml_config` folder into this deployment directory. 

## Create a model management endpoint 

Create a modelmanagement under your account. We will call this `pdmmodelmanagement`. The remaining defaults are acceptable.

`az ml account modelmanagement create --location <ACCOUNT_REGION> --resource-group <RESOURCE_GROUP> --name pdmmodelmanagement`

If you get a `ResourceGroupNotFound` error, you may need to set the correct subscription. This is typically only an issue if your Azure login connects to multiple subscritpions. 

`az account set -s '<subscription name>'`

You can find the `subscription name` or `subscription id` through the (https://portal.azure.com) under the resource group you'd like to use.

## Check environment settings

Show what environment is currently active:

`az ml env show`

If nothing is set, we setup the environment with the existing model management context first: 

` az ml env setup --location <ACCOUNT_REGION> --resource-group <RESOURCE_GROUP> --name pdmmodelmanagement`

using the same `<ACCOUNT_REGION>` and `<RESOURCE_GROUP>` in the previous section. Then set the current environment:

`az ml env set --resource-group <RESOURCE_GROUP> --cluster-name pdmmodelmanagement`

Check that the environment is now set:

`az ml env show`

## Deploy a web service 

These commands assume the current directory contains the webservice assets we created in throughout the notebooks in this scenario (`lstmscore.py`, `modellstm.json`, `modellstm.h5` and `service_schema.json`). Change to the directory where the zip file was unpacked. 

The command to create a web service (`<SERVICE_ID>`) with these operationalization assets in the current directory is:

`
az ml service create realtime -f <filename> -r <TARGET_RUNTIME> -m <MODEL_FILE> -s <SCHEMA_FILE> -n <SERVICE_ID> --cpu 0.1
`

The default cluster has only 2 nodes with 2 cores each. Some cores are taken for system components. AMLWorkbench asks for 1 core per service. To deploy multiple services into this cluster, we specify the cpu requirement in the service create command as (--cpu 0.1) to request 10% of a core. 

For this example, we will call our webservice `lstmwebservice`. This `SERVICE_ID` must be all lowercase, with no spaces:

`
az ml service create realtime -f lstmscore.py -r python -m modellstm.json -m modellstm.h5 -s service_schema.json -c webservices_conda.yaml --cpu 0.1 -n lstmwebservice
`

This command will take some time to execute. 

## Test your deployment.

Once complete, the `az ml service create` command returns sample usage commands to test the service for both PowerShell and the cmd prompt. We can test this deployment by executing these commands from the command line. 

```
> az ml service usage realtime -i lstmwebservice
Scoring URL:
    http://127.0.0.1:32770/score

Headers:
    Content-Type: application/json

Swagger URL:
    http://127.0.0.1:32770/swagger.json

Sample CLI command:
```



# Conclusion

Working through all of these notebooks, we have completed:

 * Data aquisition in `Code/1_data_aquisition.ipynb` notebook.
 * Time series feature engineering and failure labeling to predict component failures within a 7 day window in the `Code/2_feature_engineering.ipynb` notebook.
 * Model building and evaluation in the `Code/3_model_building.ipynb` notebook.
 * Deployment asset generation and model deployment in the `Code/4_operationalization.ipynb` notebook.
    
 